In [ ]:
import numpy as np
import math

#**Part 1**

In [ ]:
products_data = [
    {'product': 1, 'demand': 1000, 'common':100, 'specific_ordering_cost': 10, 'unit_cost': 50, 'holding_cost_rate': 0.2},
    {'product': 2, 'demand': 300, 'common':100, 'specific_ordering_cost': 20, 'unit_cost': 60, 'holding_cost_rate': 0.2},
    {'product': 3, 'demand': 100, 'common':100, 'specific_ordering_cost': 25, 'unit_cost': 30, 'holding_cost_rate': 0.2},
    {'product': 4, 'demand': 50, 'common':100, 'specific_ordering_cost': 25, 'unit_cost': 30, 'holding_cost_rate': 0.2}
]


In [ ]:
calculated_values = {}

### Calculate holding cost

In [ ]:
for product in products_data:
    holding_cost_per_unit = product['holding_cost_rate'] * product['unit_cost']

    calculated_values[product['product']] = {'holding_cost_per_unit': holding_cost_per_unit}

### Calculate EOQ for eqch product

In [ ]:
for product in products_data:
    # S
    total_ordering_cost = product['common'] + product['specific_ordering_cost']
    holding_cost_per_unit = calculated_values[product['product']]['holding_cost_per_unit']
    # EOQ (Q)
    eoq = math.sqrt((2 * product['demand'] * total_ordering_cost) / holding_cost_per_unit)

    # Store in dictionary
    calculated_values[product['product']]['eoq'] = eoq

### Total annual cost per unit

In [ ]:
total_cost = 0
for product in products_data:
    eoq = calculated_values[product['product']]['eoq']
    holding_cost_per_unit = calculated_values[product['product']]['holding_cost_per_unit']

    # Calculate the total annual cost (Cost_Q)
    annual_holding_cost = holding_cost_per_unit * eoq / 2
    annual_ordering_cost = product['demand'] * (product['common']+product['specific_ordering_cost']) / eoq
    total_annual_cost = annual_holding_cost + annual_ordering_cost

    # Store the total annual cost in the calculated_values dictionary
    calculated_values[product['product']]['total_annual_cost'] = total_annual_cost
    total_cost += total_annual_cost

In [ ]:
for product, values in calculated_values.items():
    print(f"Product {product}:")
    print(f"  Holding cost per unit: {values['holding_cost_per_unit']}")
    print(f"  EOQ: {values['eoq']}")
    print(f"  Total annual cost: {values['total_annual_cost']}")
print(f"Total cost: {total_cost}")

Product 1:
  Holding cost per unit: 10.0
  EOQ: 148.32396974191326
  Total annual cost: 1483.2396974191327
Product 2:
  Holding cost per unit: 12.0
  EOQ: 77.45966692414834
  Total annual cost: 929.51600308978
Product 3:
  Holding cost per unit: 6.0
  EOQ: 64.54972243679029
  Total annual cost: 387.2983346207417
Product 4:
  Holding cost per unit: 6.0
  EOQ: 45.64354645876384
  Total annual cost: 273.8612787525831
Total cost: 3073.9153138822376


#**Part 2**

In [ ]:
S = products_data[1]['common'] + sum(product['specific_ordering_cost'] for product in products_data)
n = np.sqrt(sum(product['demand']*product['holding_cost_rate'] * product['unit_cost'] for product in products_data)/(2*S))

def calculate_cost(product):
  H = product['holding_cost_rate'] * product['unit_cost']
  D = product['demand']
  Q = D / n
  holding_cost = H*Q/2
  return holding_cost

total_holding_cost = 0
for products in products_data:
  holding_cost = calculate_cost(products)
  total_holding_cost += holding_cost
  print(f"Total Holding Cost: {holding_cost:.2f}")
total_cost = S*n + total_holding_cost
print(f"Total Ording Cost: {S*n:.2f}")
print(f"Total Cost: {total_cost:.2f}")

Total Holding Cost: 787.84
Total Holding Cost: 283.62
Total Holding Cost: 47.27
Total Holding Cost: 23.64
Total Ording Cost: 1142.37
Total Cost: 2284.73


#**Part 3**

In [ ]:
def calculate_frequency(product, include_common=True):
    D = product['demand']
    S = (product['common'] if include_common else 0) + product['specific_ordering_cost']
    H = product['holding_cost_rate'] * product['unit_cost']
    frequency = D / np.sqrt((2 * D * S) / H)
    return frequency

# Calculate initial frequencies with common cost
for product in products_data:
    product['frequency'] = calculate_frequency(product, True)

# Identify the product with the highest frequency
most_frequent_product = max(products_data, key=lambda x: x['frequency'])
most_frequent_product_id = most_frequent_product['product']-1

# Recalculate frequencies without common cost and determine multiples
for i in range(4):
    if i != most_frequent_product_id:
        products_data[i]['new_frequency'] = calculate_frequency(products_data[i], False)
        products_data[i]['m'] = math.ceil(products_data[most_frequent_product_id]['frequency'] / products_data[i]['new_frequency'])

# Update the frequency for the most frequent product based on multiples
total_demand_weighted = sum(p['demand'] * p['holding_cost_rate'] * p['unit_cost'] * p.get('m', 1) for p in products_data)
total_specific_costs = sum(p['specific_ordering_cost'] / p.get('m', 1) for p in products_data)
new_frequency_most_frequent = np.sqrt(total_demand_weighted / (2 * (products_data[1]['common'] + total_specific_costs)))

# Assign new frequencies and calculate total cost
total_cost = 0
for i in range(4):
    if i == most_frequent_product_id:
        products_data[i]['final_frequency'] = new_frequency_most_frequent
    else:
        products_data[i]['final_frequency'] = new_frequency_most_frequent / products_data[i]['m']

    D, H = products_data[i]['demand'], products_data[i]['holding_cost_rate'] * products_data[i]['unit_cost']
    n = products_data[i]['final_frequency']
    total_cost += (D / n / 2) * H  # Holding cost component

total_cost += (products_data[1]['common'] + total_specific_costs) * new_frequency_most_frequent  # Adding ordering cost component
print(f"Total annual cost: {total_cost:.2f}")

Total annual cost: 2176.27
